In [1]:
import os, pickle, sys
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import glob
from prettytable import PrettyTable
from tqdm import tqdm

In [7]:
t=None
all_ds = {}
all_acc = {}
allc = {}
all_metrics = {}
all_runs = {}
metric_names = ['grad_norm', 'snip', 'grasp', 'fisher', 'jacob_cov','synflow']
for fname,rname in [('../results_release/nasbench2/nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p','CIFAR10')]:
    runs=[]
    f = open(fname,'rb')
    while(1):
        try:
            runs.append(pickle.load(f))
        except EOFError:
            break
    f.close()
    print(fname, len(runs))
    print(runs[0])
    
    all_runs[fname]=runs
    all_ds[fname] = {}
    metrics={}
    for k in metric_names:
        metrics[k] = []
    acc = []
    
    if t is None:
        hl=['Dataset']
        hl.extend(metric_names)
        t = PrettyTable(hl)
    
    for r in runs:
        for k,v in r['logmeasures'].items():
            if k in metrics:
                metrics[k].append(v)
        acc.append(r['testacc'])
        
    all_ds[fname]['metrics'] = metrics
    all_ds[fname]['acc'] = acc

    res = []
    crs = {}
    for k in hl:
        if k=='Dataset':
            continue
        v = metrics[k]
        cr = abs(stats.spearmanr(acc,v,nan_policy='omit').correlation)
        #print(f'{k} = {cr}')
        res.append(round(cr,3))
        crs[k]=cr
    
    ds = rname
    all_acc[ds]=acc
    allc[ds]=crs
    t.add_row([ds]+res)
      
    all_metrics[ds] = metrics
print(t)

../results_release/nasbench2/nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p 15625
{'i': 0, 'arch': '|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|nor_conv_1x1~0|skip_connect~1|skip_connect~2|', 'logmeasures': {'grad_norm': 3.2280678749084473, 'snip': 5.514030933380127, 'grasp': -0.1634197235107422, 'fisher': 0.004466889891773462, 'jacob_cov': -151.3307806940146, 'plain': 0.028314922004938126, 'synflow_bn': 16.958625670825334, 'synflow': 1.9405668633541168e+17}, 'trainacc': 88.91866665852865, 'valacc': 81.98266665690103, 'testacc': 81.52}
+---------+-----------+-------+-------+--------+-----------+---------+
| Dataset | grad_norm |  snip | grasp | fisher | jacob_cov | synflow |
+---------+-----------+-------+-------+--------+-----------+---------+
| CIFAR10 |   0.594   | 0.596 | 0.514 |  0.36  |   0.731   |  0.737  |
+---------+-----------+-------+-------+--------+-----------+---------+


In [34]:
# fname='../mytest/add_with_std_var.p'
# fname='../mytest/add_with_10_std_var.p'
fname='../mytest/sub_only_std_var.p'
rname='CIFAR10'
runs=[]
f = open(fname,'rb')
while(1):
    try:
        runs.append(pickle.load(f))
    except EOFError:
        break
f.close()
print(fname, len(runs))
print(runs[0])
my_metrics={}
my_metrics['synflow']=[]
my_acc=[]
# hl.extend(['synflow'])

../mytest/sub_only_std_var.p 15625
{'i': 0, 'arch': '|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|nor_conv_1x1~0|skip_connect~1|skip_connect~2|', 'logmeasures': {'synflow': -2473978824121126.5}, 'trainacc': 88.91866665852865, 'valacc': 81.98266665690103, 'testacc': 81.52}


In [35]:
runs

[{'i': 0,
  'arch': '|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|nor_conv_1x1~0|skip_connect~1|skip_connect~2|',
  'logmeasures': {'synflow': -2473978824121126.5},
  'trainacc': 88.91866665852865,
  'valacc': 81.98266665690103,
  'testacc': 81.52},
 {'i': 1,
  'arch': '|nor_conv_3x3~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|skip_connect~0|nor_conv_3x3~1|skip_connect~2|',
  'logmeasures': {'synflow': -1.1224411889452324e+31},
  'trainacc': 99.98533333333334,
  'valacc': 90.76933332438152,
  'testacc': 90.51333333333332},
 {'i': 2,
  'arch': '|avg_pool_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|avg_pool_3x3~0|avg_pool_3x3~1|avg_pool_3x3~2|',
  'logmeasures': {'synflow': -8.725932380425326e+20},
  'trainacc': 89.31466665852865,
  'valacc': 81.91066666178385,
  'testacc': 81.83999999999999},
 {'i': 3,
  'arch': '|avg_pool_3x3~0|+|skip_connect~0|none~1|+|none~0|none~1|skip_connect~2|',
  'logmeasures': {'synflow': -85.48557919968755},
  'trainacc': 91.23999998453776,
  'valacc': 84.37733332031

In [36]:
for r in runs:
    for k,v in r['logmeasures'].items():
        my_metrics[k].append(v)
    my_acc.append(r['testacc'])

In [37]:
len(acc)

15625

In [6]:
# for i in range(len(metrics['synflow'])):
#     metrics['synflow'][i]=metrics['synflow'][i].detach()
# metrics['synflow']

AttributeError: 'float' object has no attribute 'detach'

In [38]:
for i in range(len(my_metrics['synflow'])):
    my_metrics['synflow'][i]=-my_metrics['synflow'][i]

In [56]:
cr = stats.spearmanr(my_metrics['synflow'],metrics['synflow'],nan_policy='omit').correlation
print("std_var&sum:",cr)
# res.append(round(cr,3))
# t = PrettyTable(hl)
# t.add_row([rname]+res)
# print(t)

std_var&sum: 0.9992974897337751


In [50]:
acc

[81.52,
 90.51333333333332,
 81.83999999999999,
 83.88333333333334,
 84.49666666666667,
 87.78000000000002,
 88.64666666666666,
 88.77333333333333,
 88.81,
 84.11666666666666,
 10.0,
 87.64999999999999,
 10.0,
 87.83666666666666,
 88.3,
 56.19333333333333,
 89.90333333333332,
 86.19666666666667,
 87.44666666666667,
 84.35666666666667,
 86.92666666666666,
 89.14,
 85.79333333333334,
 86.14999999999999,
 81.62333333333332,
 83.87666666666667,
 86.94333333333334,
 86.55000000000001,
 82.7,
 84.46,
 88.51333333333334,
 87.14666666666666,
 89.07666666666665,
 88.14666666666666,
 88.22333333333334,
 87.67,
 87.39333333333333,
 86.55000000000001,
 88.49333333333334,
 86.70666666666666,
 71.41333333333334,
 85.72000000000001,
 90.64333333333333,
 88.37,
 75.18666666666667,
 86.47666666666667,
 86.88333333333333,
 86.71999999999998,
 82.67666666666668,
 87.78333333333335,
 89.62333333333333,
 85.27666666666666,
 90.29333333333334,
 34.196666666666665,
 87.28666666666668,
 84.14666666666666,
 90

In [40]:
my_metrics['synflow']

[2473978824121126.5,
 1.1224411889452324e+31,
 8.725932380425326e+20,
 85.48557919968755,
 8.604329875658673e+18,
 5.283116181128858e+20,
 4.569038420440091e+29,
 3.5436075994470636e+16,
 7.067903996232579e+24,
 89.93927242328984,
 0.0,
 1.0447238086760488e+31,
 0.05837616765053459,
 4.922763735142513e+17,
 6692901761024587.0,
 1198990504.9027202,
 2.8253859896950125e+30,
 5.949680666509162e+19,
 2.05808213590039e+25,
 3.3722917937575976e+16,
 3.2587734966211064e+16,
 2.9520299313918205e+23,
 1.7793867803343624e+18,
 1.1007656335638679e+23,
 2566211393206.314,
 83.74041147633297,
 3.023606060819973e+20,
 23476678307395.5,
 2719201740594955.0,
 3.5547109014326684e+16,
 5998782823262773.0,
 1.6954425011920522e+18,
 6.711619898784722e+24,
 1.0899955287308923e+24,
 4.730287897681794e+17,
 4.2725631963938406e+17,
 2.3866641368638706e+25,
 336086553515423.7,
 5.054869285527068e+31,
 304457937398193.0,
 105.67200502798931,
 1055577140.7178893,
 4.994902120447771e+29,
 1.5288231494526606e+20,


In [41]:
metrics['synflow']

[1.9405668633541168e+17,
 1.1493810330209561e+33,
 7.464802349925823e+22,
 2127.9747457035596,
 8.071480574484623e+20,
 5.1972791685016835e+22,
 4.239559914155398e+31,
 2.367273749427031e+18,
 6.434034652398955e+26,
 2459.222137292069,
 0.0,
 9.329702425332411e+32,
 0.7930882341050236,
 3.529628150594459e+19,
 4.695763596639524e+17,
 34218119120.733192,
 2.470106716357466e+32,
 5.95325322645694e+21,
 2.040165440043258e+27,
 2.1125742351819692e+18,
 2.179584045689162e+18,
 2.624854773185511e+25,
 1.2930733701769953e+20,
 1.0787323316885269e+25,
 151635627180137.75,
 2428.0724024393476,
 3.525561790242815e+22,
 2353676043474518.5,
 1.801736684407712e+17,
 2.3242344640973353e+18,
 4.240418334131682e+17,
 1.2703720196918483e+20,
 6.760274390058095e+26,
 1.2483375828322194e+26,
 3.492335615322088e+19,
 3.370147751586035e+19,
 2.0754384605133046e+27,
 1.8178002541934772e+16,
 4.888030732183157e+33,
 2.373989765914285e+16,
 3357.1353140148112,
 71074193499.31802,
 5.363979529225351e+31,
 1.36

In [54]:
new_list=[]
for i in range(len(my_metrics['synflow'])):
    if (my_metrics['synflow'][i]==0):
        new_list.append(0.0)
        print(new_list[i])
        continue
    new_list.append(metrics['synflow'][i]*my_metrics['synflow'][i])
    print(new_list[i])

4.8009213267292405e+32
1.2901126132551414e+64
6.51373605386928e+43
181911.15365877663
6.944968144783667e+39
2.745782967295518e+43
1.9370712133533707e+61
8.388689248441172e+34
4.5475139231589467e+51
221180.64975529647
0.0
9.746962251607445e+63
0.046297451715781285
1.737552545828454e+37
3.142828444530382e+33
4.102719992138931e+19
6.979004909447937e+62
3.5419955624284144e+41
4.1988280464343873e+52
7.124216757007888e+34
7.102770721750048e+34
7.748649856000316e+48
2.300877660895347e+38
1.1874314785369498e+48
3.891290740856545e+26
203327.7820745993
1.065990999677349e+43
5.52564953124747e+28
4.899285528335233e+32
8.2619815870123135e+34
2.5437348666237677e+33
2.1538427145107463e+38
4.537239211755866e+51
1.360682383633849e+50
1.6519752895801173e+37
1.4399169249835943e+37
4.953374541975066e+52
6.109382224113469e+30
2.4708356414825025e+65
7.227800275346822e+30
354755.2197822136
7.502429395284012e+19
2.679255272456614e+61
2.0811194880578727e+42
4.8862079776023264e+26
5.890720389580398e+23
1.400621

In [55]:
cr = stats.spearmanr(new_list,acc,nan_policy='omit').correlation
cr

0.7361125859472574

In [7]:
for i in range(10):
    print(metrics['synflow'][i])

-3092516598796372.5
-1.304439569130952e+31
-9.677895509958058e+20
-315.8054758905547
-8.009376242921481e+18
-4.58376048615979e+20
-3.683514235768167e+29
-4.576380767834154e+16
-6.249745787893618e+24
-332.536792627755


In [9]:
std_var=[]
sum_synflow=[]

In [10]:
for i in range(20):
    std_var.append(-metrics['synflow'][i])

In [11]:
fname='../results_release/nasbench2/nb2_cf10_seed42_dlrandom_dlinfo1_initwnone_initbnone.p'
rname='CIFAR10'
runs=[]
f = open(fname,'rb')
while(1):
    try:
        runs.append(pickle.load(f))
    except EOFError:
        break
f.close()
metrics={}
metrics['synflow']=[]
for r in runs:
    for k,v in r['logmeasures'].items():
        if k=='synflow':
            metrics[k].append(v)
for i in range(20):
    sum_synflow.append(metrics['synflow'][i])

In [12]:
print(sum_synflow)

[1.9405668633541168e+17, 1.1493810330209561e+33, 7.464802349925823e+22, 2127.9747457035596, 8.071480574484623e+20, 5.1972791685016835e+22, 4.239559914155398e+31, 2.367273749427031e+18, 6.434034652398955e+26, 2459.222137292069, 0.0, 9.329702425332411e+32, 0.7930882341050236, 3.529628150594459e+19, 4.695763596639524e+17, 34218119120.733192, 2.470106716357466e+32, 5.95325322645694e+21, 2.040165440043258e+27, 2.1125742351819692e+18]


In [13]:
print(std_var)

[tensor(-88.9505, dtype=torch.float64), tensor(nan, dtype=torch.float64), tensor(-68.4292, dtype=torch.float64), tensor(-19.3642, dtype=torch.float64), tensor(inf, dtype=torch.float64), tensor(-698.8009, dtype=torch.float64), tensor(-inf, dtype=torch.float64), tensor(96.5113, dtype=torch.float64), tensor(-inf, dtype=torch.float64), tensor(-19.4658, dtype=torch.float64), tensor(-0., dtype=torch.float64), tensor(-inf, dtype=torch.float64), tensor(-4.7942, dtype=torch.float64), tensor(106.4796, dtype=torch.float64), tensor(-74.7652, dtype=torch.float64), tensor(-52.2770, dtype=torch.float64), tensor(nan, dtype=torch.float64), tensor(-181.2379, dtype=torch.float64), tensor(-inf, dtype=torch.float64), tensor(-102.9600, dtype=torch.float64)]


In [14]:
for i in range(20):
    if sum_synflow[i]==0:
        print(0)
    else:
        print((sum_synflow[i]-100*std_var[i])/sum_synflow[i])

tensor(1.0000, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(1., dtype=torch.float64)
tensor(1.9100, dtype=torch.float64)
tensor(-inf, dtype=torch.float64)
tensor(1., dtype=torch.float64)
tensor(inf, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(inf, dtype=torch.float64)
tensor(1.7915, dtype=torch.float64)
0
tensor(inf, dtype=torch.float64)
tensor(605.4936, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(1., dtype=torch.float64)
tensor(inf, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
